Exercise: Build a DNN with 20 hidden layers of 100 neurons each (that's too many, but it's the point of this exercise). Use He initialization and the ELU activation function.

In [1]:
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dense(100,
                                 activation="elu",
                                 kernel_initializer="he_normal"))

Exercise: Using Nadam optimization and early stopping, train the network on the CIFAR10 dataset. You can load it with keras.datasets.cifar10.load_data(). The dataset is composed of 60,000 32 × 32–pixel color images (50,000 for training, 10,000 for testing) with 10 classes, so you'll need a softmax output layer with 10 neurons. Remember to search for the right learning rate each time you change the model's architecture or hyperparameters.

In [3]:
model.add(keras.layers.Dense(10, activation="softmax"))

In [4]:
optimizer = keras.optimizers.Nadam(learning_rate=5e-5)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

In [5]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.cifar10.load_data()

X_train = X_train_full[5000:]
y_train = y_train_full[5000:]
X_valid = X_train_full[:5000]
y_valid = y_train_full[:5000]

In [6]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_model.h5", save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

In [7]:
#!pip install tensorboard

In [8]:
#!pip3 show tensorboard

In [9]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [19]:
%tensorboard --logdir=c:\\users\\ma6114132\\PycharmProjects\\ml_practice\\training_deep_nn\\my_cifar10_logs --port=6003

In [12]:
model.fit(X_train, y_train, epochs=100,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)


Epoch 1/100
1407/1407 [==============================] - 13s 6ms/step - loss: 6.9431 - accuracy: 0.1715 - val_loss: 2.1839 - val_accuracy: 0.2338
Epoch 2/100
  18/1407 [..............................] - ETA: 8s - loss: 2.2029 - accuracy: 0.2118

C:\Users\ma6114132\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1407/1407 [==============================] - 9s 6ms/step - loss: 2.1122 - accuracy: 0.2323 - val_loss: 2.1016 - val_accuracy: 0.2304
Epoch 3/100
1407/1407 [==============================] - 9s 6ms/step - loss: 1.9872 - accuracy: 0.2734 - val_loss: 1.9494 - val_accuracy: 0.2730
Epoch 4/100
1407/1407 [==============================] - 9s 6ms/step - loss: 1.9056 - accuracy: 0.2994 - val_loss: 1.9386 - val_accuracy: 0.3022
Epoch 5/100
1407/1407 [==============================] - 9s 6ms/step - loss: 1.8435 - accuracy: 0.3248 - val_loss: 1.8674 - val_accuracy: 0.3106
Epoch 6/100
1407/1407 [==============================] - 9s 6ms/step - loss: 1.7912 - accuracy: 0.3472 - val_loss: 1.7869 - val_accuracy: 0.3482
Epoch 7/100
1407/1407 [==============================] - 9s 6ms/step - loss: 1.7330 - accuracy: 0.3673 - val_loss: 1.7614 - val_accuracy: 0.3502
Epoch 8/100
1407/1407 [==============================] - 8s 6ms/step - loss: 1.6951 - accuracy: 0.3840 - val_loss: 1.7225 - val_accuracy: 0.37

1407/1407 [==============================] - 8s 6ms/step - loss: 1.1536 - accuracy: 0.5868 - val_loss: 1.5640 - val_accuracy: 0.4752
Epoch 59/100
1407/1407 [==============================] - 9s 7ms/step - loss: 1.1472 - accuracy: 0.5878 - val_loss: 1.5639 - val_accuracy: 0.4854
Epoch 60/100
1407/1407 [==============================] - 9s 6ms/step - loss: 1.1430 - accuracy: 0.5908 - val_loss: 1.5545 - val_accuracy: 0.4756
Epoch 61/100
1407/1407 [==============================] - 9s 6ms/step - loss: 1.1340 - accuracy: 0.5906 - val_loss: 1.5943 - val_accuracy: 0.4696
Epoch 62/100
1407/1407 [==============================] - 8s 6ms/step - loss: 1.1300 - accuracy: 0.5957 - val_loss: 1.5556 - val_accuracy: 0.4776
Epoch 63/100
1407/1407 [==============================] - 9s 6ms/step - loss: 1.1220 - accuracy: 0.5966 - val_loss: 1.5913 - val_accuracy: 0.4752
Epoch 64/100
1407/1407 [==============================] - 9s 6ms/step - loss: 1.1155 - accuracy: 0.6005 - val_loss: 1.5977 - val_accuracy